# import, loading data

In [1]:
import random
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import os
import torch.nn.functional as F
import random, time, torchprofile
import torchvision.models as models
from tqdm import tqdm
import torch.optim as optim
from sklearn.metrics import precision_score, recall_score, f1_score

In [2]:
# 定義 CustomImageNetDataset

class CustomImageNetDataset(Dataset):
    """用於 ImageNet 圖片及其註解的數據集類別。"""
    
    def __init__(self, annotation_file, image_directory, transforms=None):
        """
        初始化數據集。
        
        參數:
            annotation_file (str): 註解文件的路徑。
            image_directory (str): 圖片目錄的路徑。
            transforms (callable, optional): 要應用於圖片的轉換。
        """
        self.image_directory = image_directory
        self.transforms = transforms
        self.annotations = self._load_annotations(annotation_file)

    def _load_annotations(self, file_path):
        """從文件加載註解。"""
        with open(file_path, 'r') as f:
            annotations = f.read().splitlines()
        return annotations

    def __len__(self):
        """返回數據集中的圖片總數。"""
        return len(self.annotations)

    def __getitem__(self, index):
        """根據索引檢索圖片及其標籤。"""
        annotation = self.annotations[index]
        img_file, label = annotation.split()
        full_img_path = os.path.join(self.image_directory, img_file)
        image = Image.open(full_img_path).convert('RGB')
        
        if self.transforms:
            image = self.transforms(image)

        return image, int(label)

In [3]:
#Data augmentation

class SelectiveChannelDrop:
    def __init__(self):
        self.possible_channels = [
            (0, 1, 2),  # RGB
            (0, 1),     # RG
            (0, 2),     # RB
            (1, 2),     # GB
            (0,),       # R
            (1,),       # G
            (2,)        # B
        ]

    def __call__(self, image):
        channels = list(image.split())
        selected_channels = random.choice(self.possible_channels)
        new_channels = [channels[i] if i in selected_channels else Image.new('L', image.size, 'black') for i in range(3)]
        output_image = Image.merge('RGB', new_channels)
        return output_image

# 配置圖像變換

basic_transforms = transforms.Compose([
    transforms.Resize(96),
    transforms.CenterCrop(96),
    transforms.ToTensor(),
])

training_transforms = transforms.Compose([
    transforms.RandomResizedCrop(96),
    transforms.RandomHorizontalFlip(1),
    transforms.RandomVerticalFlip(0.1),
    transforms.RandomPerspective(distortion_scale=0.2, p=0.2),  
    transforms.RandomRotation(15),
    SelectiveChannelDrop(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


validation_transforms = transforms.Compose([
    transforms.Resize(96),
    transforms.CenterCrop(96),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [5]:
# 把 train, validation data load進來

train_dataset = CustomImageNetDataset(annotation_file='data/images/train.txt', image_directory='data/images', transforms=training_transforms)
val_dataset = CustomImageNetDataset(annotation_file='data/images/val.txt', image_directory='data/images', transforms=validation_transforms)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

# 我的設計模型

In [4]:
#設計 DynamicConv2D 

class DynamicConv2D(nn.Module):
    def __init__(self, input_channels, output_channels, kernel_size, stride=1, padding=0, use_bias=True):
        super(DynamicConv2D, self).__init__()
        self.input_channels = input_channels
        self.output_channels = output_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.use_bias = use_bias

        # 初始化可調整的卷積核權重及偏置
        self.weights = nn.Parameter(torch.randn(output_channels, input_channels, kernel_size, kernel_size))
        if use_bias:
            self.biases = nn.Parameter(torch.randn(output_channels))
        else:
            self.register_parameter('biases', None)

    def forward(self, input_tensor):
        # 根據輸入的通道數量調整權重
        current_input_channels = input_tensor.size(1)
        if current_input_channels < self.input_channels:
            adjusted_weights = self.weights[:, :current_input_channels, :, :]
        else:
            adjusted_weights = self.weights

        # 執行卷積操作
        return F.conv2d(input_tensor, adjusted_weights, self.biases, self.stride, self.padding)


In [6]:
#把resnet18第一層改成 DynamicConv2D 

model = models.resnet18(weights=None)
model.conv1 = DynamicConv2D(input_channels=3, output_channels=64, kernel_size=7, stride=2, padding=3, use_bias=False)
model.fc = torch.nn.Linear(model.fc.in_features, 50)
model.cuda()

ResNet(
  (conv1): DynamicConv2D()
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1)

In [7]:
#定義train, validation

lr = 0.1
epochs = 30
momentum = 0.9
weight_decay = 1e-4
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)


def adjust_lr(optimizer, epoch):
    if epoch in [epochs*0.5, epochs*0.75, epochs*0.85]:
        for p in optimizer.param_groups:
            p['lr'] *= 0.1
            lr = p['lr']
        print('Change lr:'+str(lr))

def train(epoch):
    model.train()
    avg_loss = 0.
    train_acc = 0.
    adjust_lr(optimizer, epoch)
    train_loader_len = len(train_loader.dataset)
    train_loader_iter = tqdm(enumerate(train_loader), total=len(train_loader), desc="Training Epoch #{}".format(epoch))
    
    for batch_idx, (data, target) in train_loader_iter:
        data, target = data.cuda(), target.cuda()
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        avg_loss += loss.item()
        pred = output.data.max(1, keepdim=True)[1]
        train_acc += pred.eq(target.data.view_as(pred)).cpu().sum()
        loss.backward()
        optimizer.step()
        train_loader_iter.set_postfix(loss=loss.item(), accuracy=100. * train_acc.item() / train_loader_len)
    
    print('Train Epoch: {}, Loss: {:.6f}, Accuracy: {:.2f}%'.format(epoch, avg_loss / len(train_loader), 100. * train_acc / train_loader_len))



def val(epoch):
    model.eval()
    test_loss = 0.
    correct = 0
    val_loader_iter = tqdm(val_loader, total=len(val_loader), desc="Validation Epoch #{}".format(epoch))
    
    with torch.no_grad():
        for data, label in val_loader_iter:
            data, label = data.cuda(), label.cuda()
            output = model(data)
            test_loss += F.cross_entropy(output, label, reduction='sum').item()
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(label.data.view_as(pred)).cpu().sum()
            val_loader_iter.set_postfix(loss=test_loss / len(val_loader.dataset), accuracy=100. * correct.item() / len(val_loader.dataset))
    
    test_loss /= len(val_loader.dataset)
    accuracy = 100. * correct / len(val_loader.dataset)
    print('Validation Set: Average Loss: {:.4f}, Accuracy: {:.2f}%'.format(test_loss, accuracy))

    return accuracy


#開始訓練

best_val_acc = 0.
for i in range(epochs):
    train(i + 1)
    temp_acc = val(i + 1)
    if temp_acc > best_val_acc:
        best_val_acc = temp_acc
        torch.save(model.state_dict(), 'mydesign_best1.pt')
        print('Best Accuracy: {:.2f}%'.format(best_val_acc))

print('Final Best Accuracy: {:.2f}%'.format(best_val_acc))

Training Epoch #1: 100%|██████████| 990/990 [02:58<00:00,  5.55it/s, accuracy=4.65, loss=3.78] 


Train Epoch: 1, Loss: 3.815005, Accuracy: 4.65%


Validation Epoch #1: 100%|██████████| 8/8 [00:01<00:00,  7.68it/s, accuracy=7.56, loss=3.63]  


Validation Set: Average Loss: 3.6338, Accuracy: 7.56%
Best Accuracy: 7.56%


Training Epoch #2: 100%|██████████| 990/990 [02:54<00:00,  5.68it/s, accuracy=8.56, loss=3.72] 


...

Validation Set: Average Loss: 1.5511, Accuracy: 53.33%


Training Epoch #30: 100%|██████████| 990/990 [02:55<00:00,  5.63it/s, accuracy=45.9, loss=1.97]


Train Epoch: 30, Loss: 1.899953, Accuracy: 45.89%


Validation Epoch #30: 100%|██████████| 8/8 [00:01<00:00,  7.95it/s, accuracy=57.3, loss=1.45] 


Validation Set: Average Loss: 1.4464, Accuracy: 57.33%
Best Accuracy: 57.33%
Final Best Accuracy: 57.33%


In [8]:
#把 best model load 進來

model = models.resnet18(weights=None)
model.conv1 = DynamicConv2D(input_channels=3, output_channels=64, kernel_size=7, stride=2, padding=3, use_bias=False)
model.fc = torch.nn.Linear(model.fc.in_features, 50)
model.load_state_dict(torch.load("mydesign_best1.pt"))
model.cuda()

ResNet(
  (conv1): DynamicConv2D()
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1)

In [19]:
# 把 test data load進來 ＆ data transforms 



def select_channels(image, channels='RGB'):
    if channels == 'R':
        return image[0, :, :].unsqueeze(0)
    elif channels == 'G':
        return image[1, :, :].unsqueeze(0)
    elif channels == 'B':
        return image[2, :, :].unsqueeze(0)
    elif channels == 'RG':
        return image[0:2, :, :]
    elif channels == 'RB':
        return torch.stack([image[0, :, :], image[2, :, :]], dim=0)
    elif channels == 'GB':
        return image[1:, :, :]
    elif channels == 'RGB':
        return image
    else:
        raise ValueError('Invalid channel selection')


def rgb_dataloader(set_channel = 'RGB'):
    test_transforms = transforms.Compose([
        transforms.Resize(96),
        transforms.CenterCrop(96),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        transforms.Lambda(lambda x: select_channels(x, set_channel))  # Change 'RGB' to other combinations as needed
    ])
    
    test_dataset = CustomImageNetDataset(annotation_file='data/images/test.txt', image_directory='data/images', transforms=test_transforms)
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)
    return test_loader

In [20]:
#定義 test

def test(model, rgb_set):
    model.eval()
    correct = 0
    test_loader = rgb_dataloader(set_channel = rgb_set)
    test_loader_len = len(test_loader.dataset)
    test_loader_iter = tqdm(test_loader, total=len(test_loader), desc="Testing")
    
    all_preds = []
    all_targets = []
    
    start_time = time.time()
    
    with torch.no_grad():
        for data, target in test_loader_iter:
            data, target = data.cuda(), target.cuda()
            output = model(data)
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).cpu().sum()
            
            all_preds.extend(pred.cpu().numpy())
            all_targets.extend(target.cpu().numpy())
            
            test_loader_iter.set_postfix(accuracy=100. * correct.item() / test_loader_len)
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    
    accuracy = 100. * correct / test_loader_len
    
    # 計算 Precision, Recall 和 F1-score
    precision = 100. * precision_score(all_targets, all_preds, average='macro')
    recall = 100. * recall_score(all_targets, all_preds, average='macro')
    f1 = 100. * f1_score(all_targets, all_preds, average='macro')
    
    # 計算 FLOPS
    flops = torchprofile.profile_macs(model, torch.randn(1, *data.shape[1:]).cuda())
    
    return accuracy.item(), precision, recall, f1, flops, elapsed_time

#開始測試

rgb_list = ["RGB", "RG", "RB", "GB", "R", "G", "B"]
for rgb in rgb_list:
    resnet_acc, resnet_precision, resnet_recall, resnet_f1, resnet_flops, resnet_elapsed_time = test(model,rgb)
    print(f"RGB Set: {rgb:s}, Accuracy: {resnet_acc:.2f}%, Precision: {resnet_precision:.2f}%, Recall: {resnet_recall:.2f}%, F1 Score: {resnet_f1:.2f}%, FLOPS: {resnet_flops:d}, Elapsed Time: {resnet_elapsed_time:.2f} seconds")
    # resnet18_acc, resnet18_elapsed_time = test(model,rgb)
    # print(f"RGB Set: {rgb:s}, Accuracy: {resnet18_acc:.2f}%, Elapsed Time: {resnet18_elapsed_time:.2f} seconds")

Testing: 100%|██████████| 8/8 [00:01<00:00,  7.41it/s, accuracy=57.1]


RGB Set: RGB, Accuracy: 57.11%, Precision: 58.52%, Recall: 57.11%, F1 Score: 56.00%, FLOPS: 333585408, Elapsed Time: 1.08 seconds


Testing: 100%|██████████| 8/8 [00:01<00:00,  7.61it/s, accuracy=44]  


RGB Set: RG, Accuracy: 44.00%, Precision: 51.40%, Recall: 44.00%, F1 Score: 42.50%, FLOPS: 326360064, Elapsed Time: 1.05 seconds


Testing: 100%|██████████| 8/8 [00:01<00:00,  7.56it/s, accuracy=40.2]
/home/r26121073/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


...

RGB Set: R, Accuracy: 17.56%, Precision: 22.61%, Recall: 17.56%, F1 Score: 15.33%, FLOPS: 319134720, Elapsed Time: 1.02 seconds


Testing: 100%|██████████| 8/8 [00:01<00:00,  8.00it/s, accuracy=15.6]
/home/r26121073/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


RGB Set: G, Accuracy: 15.56%, Precision: 16.53%, Recall: 15.56%, F1 Score: 12.30%, FLOPS: 319134720, Elapsed Time: 1.00 seconds


Testing: 100%|██████████| 8/8 [00:00<00:00,  8.03it/s, accuracy=12.7]
/home/r26121073/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


RGB Set: B, Accuracy: 12.67%, Precision: 14.57%, Recall: 12.67%, F1 Score: 10.32%, FLOPS: 319134720, Elapsed Time: 1.00 seconds


# Resnet18 模型

In [27]:
#引入 resnet18

model = models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(model.fc.in_features, 50)
model.cuda()

/home/r26121073/.local/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/r26121073/.local/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [30]:
#定義train, validation

epochs = 30
# batch_size = 32
# test_batch_size = 32
lr = 0.1
momentum = 0.9
weight_decay = 1e-4
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)

def adjust_lr(optimizer, epoch):
    if epoch in [epochs*0.5, epochs*0.75, epochs*0.85]:
        for p in optimizer.param_groups:
            p['lr'] *= 0.1
            lr = p['lr']
        print('Change lr:'+str(lr))

def train(epoch):
    model.train()
    avg_loss = 0.
    train_acc = 0.
    adjust_lr(optimizer, epoch)
    train_loader_len = len(train_loader.dataset)
    train_loader_iter = tqdm(enumerate(train_loader), total=len(train_loader), desc="Training Epoch #{}".format(epoch))
    
    for batch_idx, (data, target) in train_loader_iter:
        data, target = data.cuda(), target.cuda()
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        avg_loss += loss.item()
        pred = output.data.max(1, keepdim=True)[1]
        train_acc += pred.eq(target.data.view_as(pred)).cpu().sum()
        loss.backward()
        optimizer.step()
        train_loader_iter.set_postfix(loss=loss.item(), accuracy=100. * train_acc.item() / train_loader_len)
    
    print('Train Epoch: {}, Loss: {:.6f}, Accuracy: {:.2f}%'.format(epoch, avg_loss / len(train_loader), 100. * train_acc / train_loader_len))



def val(epoch):
    model.eval()
    test_loss = 0.
    correct = 0
    val_loader_iter = tqdm(val_loader, total=len(val_loader), desc="Validation Epoch #{}".format(epoch))
    
    with torch.no_grad():
        for data, label in val_loader_iter:
            data, label = data.cuda(), label.cuda()
            output = model(data)
            test_loss += F.cross_entropy(output, label, reduction='sum').item()
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(label.data.view_as(pred)).cpu().sum()
            val_loader_iter.set_postfix(loss=test_loss / len(val_loader.dataset), accuracy=100. * correct.item() / len(val_loader.dataset))
    
    test_loss /= len(val_loader.dataset)
    accuracy = 100. * correct / len(val_loader.dataset)
    print('Validation Set: Average Loss: {:.4f}, Accuracy: {:.2f}%'.format(test_loss, accuracy))

    return accuracy


best_val_acc = 0.
for i in range(epochs):
    train(i + 1)
    temp_acc = val(i + 1)
    if temp_acc > best_val_acc:
        best_val_acc = temp_acc
        torch.save(model.state_dict(), 'resnet18_best.pt')
        print('Best Accuracy: {:.2f}%'.format(best_val_acc))

print('Final Best Accuracy: {:.2f}%'.format(best_val_acc))
    

Training Epoch #1: 100%|██████████| 990/990 [02:59<00:00,  5.53it/s, accuracy=3.6, loss=3.88]  


Train Epoch: 1, Loss: 3.877870, Accuracy: 3.60%


Validation Epoch #1: 100%|██████████| 8/8 [00:00<00:00,  8.17it/s, accuracy=8.22, loss=3.67] 


Validation Set: Average Loss: 3.6710, Accuracy: 8.22%
Best Accuracy: 8.22%


Training Epoch #2: 100%|██████████| 990/990 [02:59<00:00,  5.51it/s, accuracy=6.26, loss=3.67] 


...

Validation Set: Average Loss: 1.7073, Accuracy: 48.89%


Training Epoch #30: 100%|██████████| 990/990 [02:57<00:00,  5.56it/s, accuracy=39.8, loss=1.96]


Train Epoch: 30, Loss: 2.155499, Accuracy: 39.77%


Validation Epoch #30: 100%|██████████| 8/8 [00:01<00:00,  7.89it/s, accuracy=52.4, loss=1.65] 


Validation Set: Average Loss: 1.6484, Accuracy: 52.44%
Best Accuracy: 52.44%
Final Best Accuracy: 52.44%


In [31]:
#把 best model load 進來

model = models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(model.fc.in_features, 50)
model.load_state_dict(torch.load("resnet18_best.pt"))
model.cuda()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [33]:
# 把 test data load進來 ＆ data transforms 

test_augmentations = transforms.Compose([
    transforms.Resize(96),
    transforms.CenterCrop(96),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

test_dataset = CustomImageNetDataset(annotation_file='data/images/test.txt', image_directory='data/images', transforms=test_augmentations)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [36]:
#定義test

def test(model):
    model.eval()
    correct = 0
    test_loader_len = len(test_loader.dataset)
    test_loader_iter = tqdm(test_loader, total=len(test_loader), desc="Testing")
    
    all_preds = []
    all_targets = []
    
    start_time = time.time()
    
    with torch.no_grad():
        for data, target in test_loader_iter:
            data, target = data.cuda(), target.cuda()
            output = model(data)
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).cpu().sum()
            
            all_preds.extend(pred.cpu().numpy())
            all_targets.extend(target.cpu().numpy())
            
            test_loader_iter.set_postfix(accuracy=100. * correct.item() / test_loader_len)
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    
    accuracy = 100. * correct / test_loader_len
    
    # 計算 Precision, Recall 和 F1-score
    precision = 100. * precision_score(all_targets, all_preds, average='macro')
    recall = 100. * recall_score(all_targets, all_preds, average='macro')
    f1 = 100. * f1_score(all_targets, all_preds, average='macro')
    
    # 計算 FLOPS
    flops = torchprofile.profile_macs(model, torch.randn(1, *data.shape[1:]).cuda())
    
    return accuracy.item(), precision, recall, f1, flops, elapsed_time

#開始測試

resnet_acc, resnet_precision, resnet_recall, resnet_f1, resnet_flops, resnet_elapsed_time = test(model)
print(f"Accuracy: {resnet_acc:.2f}%, Precision: {resnet_precision:.2f}%, Recall: {resnet_recall:.2f}%, F1 Score: {resnet_f1:.2f}%, FLOPS: {resnet_flops:d}, Elapsed Time: {resnet_elapsed_time:.2f} seconds")

Testing: 100%|██████████| 8/8 [00:00<00:00,  8.14it/s, accuracy=51.1]


Accuracy: 51.11%, Precision: 52.57%, Recall: 51.11%, F1 Score: 49.78%, FLOPS: 333585408, Elapsed Time: 0.98 seconds
